In [1]:
import numpy as np 
import websocket
import json
import pandas as pd

In [2]:

# Your Deribit credentials
api_key = 'x_uXagP8'
api_secret = 'ygSW1HwArk2tVnDU_FpByAZLMPJkaUx51Y56NS24DrI'

## Defining functions

In [3]:
from scipy.stats import norm
#Lets create a delta function
def delta(S, K, r, T, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        delta = np.exp(-r * T) * norm.cdf(d1)
    elif option_type == 'put':
        delta = np.exp(-r * T) * (norm.cdf(d1) - 1)
    return delta

In [4]:
import requests
import time
from datetime import datetime, timedelta

def get_last_trades(currency, kind, start_time, end_time, count=1000):
    # Convert timestamps to milliseconds since epoch
    end_timestamp = int(time.mktime(end_time.timetuple()) * 1000)
    start_timestamp = int(time.mktime(start_time.timetuple()) * 1000)


    # Define a message to get option trading information
    options_message = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "public/get_last_trades_by_currency_and_time",
        "params": {
            'currency': currency,
            'kind': kind,
            'start_timestamp': start_timestamp,
            'end_timestamp': end_timestamp,
            'count': count,
            'sorting': 'asc'
        }
    }

    # Send the message to get options information
    ws.send(json.dumps(options_message))
    options_response = ws.recv()
    print(options_response)

    # Close the WebSocket connection

    return options_response



In [5]:

def get_calls(options_response):
    parsed_response = json.loads(options_response)
    df_4 = pd.json_normalize(parsed_response['result']['trades'])
    df_4['timestamp'] = pd.to_datetime(df_4['timestamp'], unit='ms')
    calls = df_4[df_4['instrument_name'].str.endswith('C')].copy()
    calls.loc[:,'maturity'] = calls['instrument_name'].apply(lambda x: x.split('-')[1])
    calls.loc[:,'strike'] = calls['instrument_name'].apply(lambda x: x.split('-')[2])
    calls.loc[:,'strike'] = calls['strike'].apply(lambda x: x[:-1])
    calls.loc[:,'strike'] = calls['strike'].astype(float)
    calls.loc[:,'maturity'] = pd.to_datetime(calls['maturity'],format='%d%b%y')
    calls.loc[:,'Date'] = calls['timestamp']
    #Lets create a time to maturity column, and also a delta column
    calls.loc[:,'time_to_maturity'] = (pd.to_datetime(calls['maturity']) - pd.to_datetime(calls['Date'])).dt.days
    #calls.loc[:,'delta'] = delta(S=calls['index_price'], K=calls['strike'], r=0.01, T=calls['time_to_maturity']/365, sigma=calls['iv'], option_type='call')
    return calls

## Creating the Start and endtime

In [6]:
import requests
import time
from datetime import datetime, timedelta
# Calculate the timestamps for the last month
#End time here is now! But could be some other in the past!
end_time = datetime.now()
#Lets near the end time to the nearst hour
end_time = end_time.replace(minute=0, second=0, microsecond=0)
#Lets reduce the start time to 1 hour before
start_time = end_time - timedelta(hours=1) #Just 1 hour before

#Lets create this as a function
def time_lapse(start_time, end_time):
    # Convert timestamps to milliseconds since epoch
    end_timestamp = int(time.mktime(end_time.timetuple()) * 1000)
    start_timestamp = int(time.mktime(start_time.timetuple()) * 1000)
    return start_timestamp, end_timestamp

## Making the GET function

In [7]:

# Establish a connection to Deribit's WebSocket API
ws = websocket.create_connection('wss://www.deribit.com/ws/api/v2')
#Websocket connection is the same as the API connection, but we need to send a message to authenticate

# Define your authentication message
auth_message = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "public/auth",
    "params": {
        "grant_type": "client_credentials",
        "client_id": api_key,
        "client_secret": api_secret
    }
}

# Send the authentication message
ws.send(json.dumps(auth_message)) #We need to send the message as a string
response = ws.recv()
print(response)


{"jsonrpc":"2.0","id":1,"result":{"token_type":"bearer","scope":"name:MQE block_trade:read connection trade:read wallet:read account:read custody:read mainaccount","refresh_token":"1747352379608.1YzfwKoa.zaz8ZtjtDSOIml2UXWLD3obpgA-qtpc6YF_voWCPURjAtpfctKMReNBAB3O_4_Nc0ajTb4BXmdQiU7xTge-aXe643D5SXkq40eUJvjcx25TKmpGlXlv8ldx44FGRET38-SBGrjvNsb5Q3RTFjjoPw4DegCl96R-DEo0xxD0xAIuG0Aot0Yk-yATRCMDNKrvuDtmI_nS8d5HHJ0AMjY9OdTXKF7TyYA-grHvYS8pzgkPT-mCDjrA-JuzIdUuw9LsVGgZHFlYcqecMxY6PL_VGE4ZeD-gU8qBbtxPjcrIJfKlRsXC_qj_SCzA0WDk","expires_in":31536000,"access_token":"1747352379608.1a6xvMpg.rOMNsLwQ5bncmKy3p2fawIITKnrSLC1t5M0VemGC3hpNNgvJDt-bKJeGLkB_TLBS36oPFaWyPAarPGPxVgPdn_XSf6BWDg-B3OICiMBHwneQpZ1gD2f0JDBj3fV38pLrx_peL3qLskHsAd6CxdSYDR0T-g5Pp8EUqubqGzsXW-Dq8vrN4ryVz5k9i7LZbokFLts4D3sljYHvdhiE5Zd92cyw2puShZ_xeg96zUxvm5RUu48z1RrYCXuUebukjgFObbGkKQsNmYpXWbtPpIfm1vHzHwlu0O5wXQ349_A9apqMhFlD4KN23rb-aIhe"},"usIn":1715816379608054,"usOut":1715816379608325,"usDiff":271,"testnet":false}


## Getting the data
Creating the websocket conection and getting a loop
Problem: After 7 days of downloading the results starts to come empty (result:{}). I dont know if this is because im blocked or because the API doesnt get more than 7 days or that the API has a limit of requests. No idea.

In [13]:
#Lets create a loop to get the calls for the last day, looping every hour
calls = []
empty = []

# Establish a connection to Deribit's WebSocket API
ws = websocket.create_connection('wss://www.deribit.com/ws/api/v2')

# Define your authentication message
auth_message = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "public/auth",
    "params": {
        "grant_type": "client_credentials",
        "client_id": api_key,
        "client_secret": api_secret
    }
}

# Send the authentication message
ws.send(json.dumps(auth_message))
response = ws.recv()
print(response)
for i in range(24*90): #Get from the last 90 days
    start_time = end_time - timedelta(hours=1)
    start_timestamp, end_timestamp = time_lapse(start_time, end_time)
    response = get_last_trades('BTC', 'option', start_time, end_time)
    # Lets wait 1 second before the next call
    test = json.loads(response)["result"]["trades"]
    if not test:
        print("Empty")
        empty.append(start_time)
    else:
        calls.append(get_calls(response))
        end_time = start_time
    time.sleep(1)
    
ws.close()

{"jsonrpc":"2.0","id":1,"result":{"token_type":"bearer","scope":"name:MQE block_trade:read connection trade:read wallet:read account:read custody:read mainaccount","refresh_token":"1747343387485.1T-_kwk6.0XNZKHRg_Fwc3kQ4rEVnKFK8YNrwXXV3ZQjVKEQXwFmkLRS5_FYRyr3ILcwu74WwxMUoxTDlYqoif2EW9l9O3f6s8eRH_qtClmRHVTw9BFh_TpE_piYDRRcB_cxSjoR5NHRUtDFG4AXTrRQCR_CXOlWBgJUuP5vJjv2b8nUvWsSLG3fLbKGDdGGgvQ9tm31gQUELElo8A3gHSOpgLC37pOIpxyKtFtUlcWoFC-6-NovJobW81J4EmXrXOPEfDzm-RBAZJVgp1WJdQzk98rslmPPbEhBQDD0d_hZsmSd12cLMAQFxXztPGQY","expires_in":31536000,"access_token":"1747343387485.1Q_tuWBQ.V_j0vkrqeJIxaljlC5Q_tWbYBrxhB0zJfwYGIUG5c1FB2umrkR0ZNCNFJojcjGZq9DSGTGiswcFjrfY439svmuXoVsT1i1gAll1wjWGZ9b5bw2bOh8F13YDi9_7j7wkUnek-W1yTOCyj1hozDBeJJOJSFS2vJDn_Ijau0V8ZxbxluI0NZIN5XqXZX1NskuTHZn0t0KEqTYIffMYqX7UH0OJ6NTeIt3ZgsuMHYXZGtRHmJXsMiFPHSDj4OWUrroGOeKTA_2F_cMmOzy4wGmwrIxRiMXP7prK-nL8YQA792uArmBqHm0RMlC49"},"usIn":1715807387485591,"usOut":1715807387485857,"usDiff":266,"testnet":false}
{"jsonrpc":"2.0","id":2,"res

In [14]:
calls_df = pd.concat(calls)
calls_df.head()
#Lets save calls_df to csv
calls_df

,trade_id,contracts,instrument_name,tick_direction,trade_seq,mark_price,amount,index_price,direction,price,iv,timestamp,block_trade_leg_count,block_trade_id,combo_id,maturity,strike,Date,time_to_maturity
0,301609578,1.0,BTC-17MAY24-63000-C,0,1383,0.036082,1.0,65116.60,sell,0.0360,52.69,2024-05-15 18:00:00.050,NaN,NaN,NaN,2024-05-17 00:00:00,6300.0,2024-05-15 18:00:00.050,1
1,301609588,2.5,BTC-31MAY24-64000-C,0,1472,0.054907,2.5,65113.20,sell,0.0550,53.29,2024-05-15 18:00:03.499,NaN,NaN,NaN,2024-05-31 00:00:00,6400.0,2024-05-15 18:00:03.499,15
2,301609590,1.4,BTC-18MAY24-72000-C,1,6,0.000586,1.4,65108.26,sell,0.0005,60.44,2024-05-15 18:00:04.735,NaN,NaN,NaN,2024-05-18 00:00:00,7200.0,2024-05-15 18:00:04.735,2
7,301609606,0.4,BTC-24MAY24-69000-C,1,329,0.012157,0.4,65091.24,sell,0.0120,52.64,2024-05-15 18:00:09.981,NaN,NaN,NaN,2024-05-24 00:00:00,6900.0,2024-05-15 18:00:09.981,8
10,301609635,2.0,BTC-16MAY24-65000-C,2,269,0.007775,2.0,65076.50,buy,0.0075,42.94,2024-05-15 18:00:19.913,NaN,NaN,NaN,2024-05-16 00:00:00,6500.0,2024-05-15 18:00:19.913,0


In [ ]:
calls_df.to_csv('calls_df.csv', index=False)